# A Column Generation Example -- Metal Cutting

Suppose we stock standard-length solid metal blocks that are used to craft android robots. The stock length is 19 meters.

We have an order for

* 12 lengths of 4m
* 15 lengths of 5m
* 22 lengths of 6m
 
How should we cut our standard length metal blocks to meet the demand and minimize the number of standard lengths we need to use?


## First Modeling Approach

In [5]:
using JuMP, Gurobi

scale = 100 # factor to increase/decrease size of problem

N = 16*scale  # upper bound on number of pipes needed

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m,"OutputFlag",0)

@variable(m, x[1:3,1:N] >= 0, Int) # times to cut each pattern 1,2,3 from given block 1-N
@variable(m, z[1:N], Bin) # use block 1-N
for j = 1:N
    @constraint(m, 4x[1,j] + 5x[2,j] + 6x[3,j] <= 19) # only use patterns that fit on block
end
@constraint(m, sum(x[1,j] for j=1:N) >= 12*scale) # meet demand for pattern 1
@constraint(m, sum(x[2,j] for j=1:N) >= 15*scale) # meet demand for pattern 2
@constraint(m, sum(x[3,j] for j=1:N) >= 22*scale) # meet demand for pattern 3
for j = 1:N
    # upper bounds on how many of each pattern can be used (0 if z=0)
    @constraint(m, x[1,j] <= 4z[j]) 
    @constraint(m, x[2,j] <= 3z[j])
    @constraint(m, x[3,j] <= 3z[j])
end

# symmetry-breaking
for j = 1:N-1
    @constraint(m, z[j] >= z[j+1])
end

# minimize the total blocks used
@objective(m, Min, sum(z[j] for j=1:N))

@time(optimize!(m))



Academic license - for non-commercial use only
Academic license - for non-commercial use only
236.152520 seconds (338.75 k allocations: 16.828 MiB, 0.04% gc time)


## Second Modeling Approach (Column Generation)

In [14]:
# all the columns:
A = [ 0 0 1 0 2 1 2 3 4
      0 1 0 2 1 2 2 1 0
      3 2 2 1 1 0 0 0 0 ]

scale = 100000000

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m,"OutputFlag",0)

@variable(m, x[1:9] >= 0, Int)
@constraint(m, A*x .>= [12;15;22]*scale)
@objective(m, Min, sum(x))
@time(optimize!(m))


Academic license - for non-commercial use only
Academic license - for non-commercial use only
  0.007935 seconds (1.01 k allocations: 64.656 KiB)
